## Positive Degree Days

[Calov and Greeve (2005)](https://www.cambridge.org/core/journals/journal-of-glaciology/article/semianalytical-solution-for-the-positive-degreeday-model-with-stochastic-temperature-variations/6FD568E1D92AA7B21C25E34B012790E1): 
$$
PDD = \int\limits_{0}^{A} \left[ 
\frac{\sigma}{\sqrt{2\pi}} 
\exp \left(  -\frac{T_{\rm ac}(t)^2}{2\sigma^2}\right) 
+ \frac{T_{\rm ac}(t)}{2}  \rm{erfc}  \left( -\frac{T_{\rm ac}(t)}{\sqrt{2}\sigma} \right)
 \right] \rm{d} t
$$

In [ ]:
import sys
import numpy as np 
import scipy.special as sp
import matplotlib.pyplot as plt 

sys.path.append('../../src/thermal')

from utils import * 

def classic_PDDs(T, T_m=0.0): 
    return np.greater(T, T_m) * T

def clavo_greev_PDDs(T, σ=0.0): 
    import scipy.special as sp

    with np.errstate(divide='ignore', invalid='ignore'):
        T_norm = T / (np.sqrt(2)*σ) 
        
    clavo_greev = σ / np.sqrt(2*np.pi) * np.exp(-T_norm**2) + T/2*sp.erfc(-T_norm)
    
    return clavo_greev

In [ ]:
# default air temperature parameters from KMR data
temp_params = dict(α       = 10.8, 
                   dTdz    = 6.5E-3, 
                   z_ref   = 2193.0, 
                   T_mean  = -6.02, 
                   T_peak  = 196, 
                   T_σ     = ( 8.29376332e-05, -3.45256005e-02, 6.31076200e+00))

# initialize the AirTemp class with the default parameters
AirTemp  = surface_AirTemp(**temp_params)

In [ ]:
doy  = np.arange(0,365)
Z    = np.linspace(2000,3000)
std  = np.polyval(temp_params['T_σ'], doy)[:,None]

# temp = AirTemp(Z, T_σ=0.0)
temp = AirTemp(Z)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

ax.plot(Z, classic_PDDs(temp).sum(0), 
       label='Classic PDDs')

ax.plot(Z, clavo_greev_PDDs(temp, σ=0.0).sum(0), 
       label=r'Calvo and Greeve w/ $\sigma=0$', 
       ls = ':')

ax.plot(Z, clavo_greev_PDDs(temp, σ=std).sum(0), 
       label=r'Calvo and Greeve w/ $\sigma(d)$')

ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

ax.plot(doy, classic_PDDs(temp).sum(1), 
       label='Classic PDDs')
ax.plot(doy, clavo_greev_PDDs(temp, σ=0.0).sum(1), 
       label=r'Calvo and Greeve w/ $\sigma=0$', 
       ls = ':')
ax.plot(doy, clavo_greev_PDDs(temp, σ=std).sum(1), 
       label=r'Calvo and Greeve w/ $\sigma(d)$', 
       zorder=0)

ax.legend(fontsize='x-small')


ax2 = ax.twinx()

ax2.plot(doy, temp.mean(1), 
         label='$T(d)$', c='k', lw=0.5)

ax2.axhline(0.0, ls=':', c='k', lw=0.5)

ax2.legend(loc=2)



ax.set_xlabel('Day of Year')
ax.set_ylabel('Positive Degree Days [$^\circ$C d]')
ax2.set_ylabel('Mean Air Temperature [$^\circ$ C]')

In [ ]:
print(classic_PDDs(temp).sum())
print(clavo_greev_PDDs(temp).sum())
print(clavo_greev_PDDs(temp, σ=std).sum())

In [ ]:
doy  = np.arange(0,365)
Z    = np.linspace(2000,3000)
std  = np.polyval(temp_params['T_σ'], doy)[:,None]

# initialize the AirTemp class with the default parameters
AirTemp  = surface_AirTemp(**temp_params)
temp = AirTemp(Z, T_σ=0.0)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

ax.plot(doy, classic_PDDs(temp).sum(1), 
       label='Classic PDDs')
ax.plot(doy, clavo_greev_PDDs(temp, σ=0.0).sum(1), 
       label=r'Calvo and Greeve w/ $\sigma=0$', 
       ls = ':')
ax.plot(doy, clavo_greev_PDDs(temp, σ=std).sum(1), 
       label=r'Calvo and Greeve w/ $\sigma(d)$', 
       zorder=0)

ax.legend( facecolor='w')


ax2 = ax.twinx()

ax2.plot(doy, temp.mean(1), 
         label='$T(d)$', c='k', lw=0.5)

ax2.axhline(0.0, ls=':', c='k', lw=0.5)

ax2.legend(loc=2)



ax.set_xlabel('Day of Year')
ax.set_ylabel('Positive Degree Days [$^\circ$C d]')
ax2.set_ylabel('Mean Air Temperature [$^\circ$ C]')

In [ ]:
print(classic_PDDs(temp).sum())
print(clavo_greev_PDDs(temp).sum())
print(clavo_greev_PDDs(temp, σ=std).sum())

In [ ]:
# def clavo_greev_PDDs(T, σ=0.0): 
#     import scipy.special as sp

#     with np.errstate(divide='ignore', invalid='ignore'):
#         T_norm = T / (np.sqrt(2)*σ) 
        
#     # clavo_greev = σ / np.sqrt(2*np.pi) * np.exp(-T_norm**2) + T/2*sp.erfc(-T_norm)
#     clavo_greev = T_norm
#     return clavo_greev

Couple PDDs to KMR results? 

In [ ]:
from scipy import interpolate

# load South Glacier geometry 
glc1a_x  = np.loadtxt('../../input_data/topography/glc1-a_surf.dat')[:,0]
glc1a_zs = np.loadtxt('../../input_data/topography/glc1-a_surf.dat')[:,1]
glc1a_zb = np.loadtxt('../../input_data/topography/glc1-a_bed.dat' )[:,1]

# load KMR mass balance results
kmr_coefs = np.loadtxt("../../input_data/mass_balance/KMR_MB_k_2_s_2500_coefs.dat")
kmr_knots = np.loadtxt("../../input_data/mass_balance/KMR_MB_k_2_s_2500_knots.dat")

# pack up the knots and coefs into tck tuple
kmr_tck = (kmr_knots, kmr_coefs, 2)
# predict the mass balance from the kmr spline
kmr_mb  = interpolate.splev(glc1a_zs, kmr_tck)

In [ ]:
std  = np.polyval(temp_params['T_σ'], doy)[:,None]

# initialize the AirTemp class with the default parameters
AirTemp  = surface_AirTemp(**temp_params)
temp = AirTemp(glc1a_zs, T_σ=0.0)

glc1a_PDDs = clavo_greev_PDDs(temp, std) #units:[C+ d] [365, NX]

daily_melt  = 3e-3 * glc1a_PDDs
yearly_melt = daily_melt.sum(0)

yearly_accum = kmr_mb + yearly_melt
daily_accum  = yearly_accum * (np.ones((365,1))/365)

In [ ]:
thick     = 10.0 #[m]
thick_ref = 10.0
# pick an index of interest
idx = 100 

# get the nodal metl and accumulation
nodal_melt  = daily_melt[:,idx]
nodal_accum = daily_accum[:, idx]

# yealry ammount of snow on the surface
yearly_snow = np.maximum((nodal_accum - nodal_melt).sum(), 0.0)


for i in range(0,365): 
    
    daily_snow = np.maximum(yearly_snow + nodal_accum[i] - nodal_melt[i], 0.0)
    
    old_zsurf      = glc1a_zs[idx] + (thick-thick_ref)

    # Calculate the new thickness [m] based on the mass balance?
    thick = thick + (nodal_accum[i] - nodal_melt[i] - (daily_snow/365))
    
    # if column less than 10 m (i.e. net abalation has occured), set back to 10 m
    if thick < thick_ref: thick = thick_ref
    
    zsurf = glc1a_zs[idx]+(thick-thick_ref)
    
    plt.scatter(i, zsurf-glc1a_zs[idx])

In [ ]:
NY = 1000 // 2

In [ ]:
1000 // 2

In [ ]:
plt.plot(glc1a_zs, kmr_mb)
# plt.plot(glc1a_zs, glc1a_melt)
plt.plot(glc1a_zs, kmr_mb + glc1a_melt)

In [ ]:
plt.imshow((kmr_mb + 3e-3 *glc1a_PDDs)), plt.colorbar()